In [6]:
from qem.image_fitting import ImageModelFitting
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
%matplotlib qt

# file = '/home/zzhang/OneDrive/code/qem/data/STO/adf_average_STO.txt'

# image = np.loadtxt(file)
# file = '/home/zzhang/OneDrive/data/Nico/28.03.2024-STOonSTO21mrad/83-100/aligned_hdf5/1859 HAADF 33.7 nm.emd.hspy'
file ='/home/zzhang/OneDrive/data/Nico/28.03.2024-STOonSTO21mrad/84-100/aligned_hdf5/1932 HAADF 33.7 nm.emd.hspy'

s = hs.load(file)
image = s.data
# image = np.load(file)
# dx= 0.1645429228960236	
dx = s.axes_manager[1].scale *10
# dx = 0.36254824166962796
model=ImageModelFitting(image, dx=dx,units='A')

# get the unique coordinates by row
# input_coordinates = np.unique(input_coordinates, axis=0)

In [7]:
model.plot()

In [8]:
model.find_peaks(min_distance=10,threshold_abs=11600,threshold_rel=0.05)

# model.select_region(invert_selection=False)

Updated peak locations.


array([[1675.,  921.],
       [ 703.,  359.],
       [ 316., 1799.],
       ...,
       [ 350.,   50.],
       [ 479.,   43.],
       [ 245.,   29.]])

In [9]:
cif_file = '/home/zzhang/OneDrive/code/qem/data/STO/SrTiO3_mp-5229_conventional_standard.cif'
model.map_lattice(cif_file=cif_file,elements=['Sr','Ti'],min_distance=20, reciprocal=True)

Origin selected: [ 776. 1580.]
Vector a selected: [25.  2.]
Vector b selected: [  1. -23.]
In pixel: Origin: [ 776. 1580.], Vector a: [25.  2.], Vector b: [  1. -23.]
In space: Origin: [127.68530817 259.97781818] A, Vector a: [4.11357307 0.32908585] A, Vector b: [ 0.16454292 -3.78448723] A
Origin selected: [1024. 1024.]
Vector a selected: [84.  0.]
Vector b selected: [ 0. 84.]
In pixel: Origin: [1024. 1024.], Vector a: [84.  0.], Vector b: [ 0. 84.]
In space: Origin: [3.03872079 3.03872079] 1/A, Vector a: [0.24927006 0.        ] 1/A, Vector b: [0.         0.24927006] 1/A
FFT real a: [24.38095238  0.        ] pixel, Real b: [ 0.         24.38095238] pixel


In [5]:
model.add_or_remove_peaks(min_distance=10)

Updated peak locations.


array([[  13.80952381,  176.80952381],
       [  14.        ,  201.61904762],
       [  20.19047619, 1878.19047619],
       ...,
       [2043.19047619,  469.19047619],
       [2043.19047619,  543.19047619],
       [2043.19047619,  592.19047619]])

In [10]:
model.select_region()

INFO:root:Selected 13040 atoms out of 13485 atoms.


In [11]:
model.fit_background = True
params = model.init_params()
model.fit_random_batch(params, batch_size=1000, maxiter=50, tol=1e-3, plot=False, step_size=0.01,verbose=False,num_epoch=3)

Fitting random batch:   0%|          | 0/14 [00:00<?, ?it/s]INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
Fitting random batch: 100%|██████████| 14/14 [01:26<00:00,  6.16s/it]
INFO:root:Convergence rate for pos_x = 0.5694115048363528
INFO:root:Convergence rate for pos_y = 0.5401146298363528
INFO:root:Convergence rate for height = 0.0006984313479857043
INFO:root:Convergence rate for sigma = 0.03785689274926218
INFO:root:Convergence not reached
Fitting random batch: 100%|██████████| 14/14 [01:19<00:00,  5.71s/it]
INFO:root:Convergence rate for pos_x = 0.543701171875
INFO:root:Convergence rate for pos_y = 0.56201171875
INFO:root:Convergence rate for height = 0.0006855097198954109
INFO:root:Convergence rate for sigma = 0.034019972086610645
INFO

{'pos_x': array([  13.7669096 ,   13.75517845,   14.39914894, ..., 2040.05090332,
        2039.81433105, 2039.75964355]),
 'pos_y': array([ 762.6572876 ,  492.65628052,  614.66326904, ..., 1946.03234863,
        1970.37731934, 1994.77832031]),
 'height': array([ 859.78621996,  887.97166786,  863.51520764, ..., 1217.32249751,
        1203.69030282, 1197.64950082]),
 'sigma': array([4.57701059, 4.86537352]),
 'background': 11897.344978568384}

In [12]:
model.plot_fitting()

In [14]:
plt.figure()
plt.hist(model.volume[model.atom_types==0], bins=100, alpha=0.5,label='Sr')
plt.hist(model.volume[model.atom_types==1], bins=100, alpha=0.5,label='Ti')
plt.legend()

In [16]:
model.plot_scs()

In [15]:
model.plot_scs_histogram()